# Monodepth Estimation with DeGirum

This tutorial demonstrates Monocular Depth Estimation with MidasNet in PySDK using OpenVINO. Model information can be found [here](https://docs.openvino.ai/2024/omz_models_model_midasnet.html).

### What is Monodepth?
Monocular Depth Estimation is the task of estimating scene depth using a single image. It has many potential applications in robotics, 3D reconstruction, medical imaging and autonomous systems. This tutorial uses a neural network model called [MiDaS](https://github.com/intel-isl/MiDaS), which was developed by the [Embodied AI Foundation](https://www.embodiedaifoundation.org/). See the research paper below to learn more.

R. Ranftl, K. Lasinger, D. Hafner, K. Schindler and V. Koltun, ["Towards Robust Monocular Depth Estimation: Mixing Datasets for Zero-shot Cross-dataset Transfer,"](https://ieeexplore.ieee.org/document/9178977) in IEEE Transactions on Pattern Analysis and Machine Intelligence, doi: `10.1109/TPAMI.2020.3019967`.

### Install requirements

In [ ]:
# make sure PySDK and degirum-tools is installed
!pip show degirum || pip install degirum
!pip show degirum-tools || pip install degirum-tools

### Imports

In [ ]:
import degirum as dg
import degirum_tools as dgtools
from PIL import Image
from depth_postprocessor import DepthResults

## Monodepth on an Image

In [ ]:
hw_location = dg.CLOUD
model_name = 'resnext101_midas_depth--256x256_float_openvino_cpu_1'
model_zoo_url = 'https://cs.degirum.com/degirum/openvino_demos'
image_url = "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/data/data/image/coco_bike.jpg"

zoo = dg.connect(hw_location, model_zoo_url, dgtools.get_token())
model = zoo.load_model(model_name, image_backend="pil")    # use PIL as image backend to show results in-cell
model.custom_postprocessor = DepthResults

results = model(image_url)

# Place image and depth map side by side
compound_image = Image.new('RGB', (results.image.size[0] * 2, results.image.size[1]))
compound_image.paste(results.image)
compound_image.paste(results.image_overlay, (results.image.size[0], 0))
compound_image


## Monodepth on a Video

In [ ]:
# Video source: https://www.youtube.com/watch?v=fu1xcQdJRws (Public Domain)
video_source = "https://storage.openvinotoolkit.org/repositories/openvino_notebooks/data/data/video/Coco%20Walking%20in%20Berkeley.mp4"

zoo = dg.connect(hw_location, model_zoo_url, dgtools.get_token())
model = zoo.load_model(model_name)
model.custom_postprocessor = DepthResults

with dgtools.Display("AI Camera") as display:
    # AI prediction loop
    # Press 'x' or 'q' to stop
    # Drag zone by left mouse button to move zone
    # Drag zone corners by right mouse button to adjust zone shape
    for inference_result in dgtools.predict_stream(
        model, video_source
    ):
        display.show(inference_result)